### Log into ArcGIS Online

In [2]:
from arcgis.gis import GIS
from datetime import datetime

gis = GIS("home")

### WTF is the user.lastLogin property not just a datetime object???
Oh well we'll deal with the int...

In [3]:
# Get days since last login from datetime object
def get_time_delta(last_login):

    # user property is a Unix timestamp in miliseconds; convert to datetime object
    login_datetime = datetime.fromtimestamp(last_login / 1000)

    # Get time delta in days since last login
    delta_days = (datetime.now() - login_datetime).days

    return delta_days

### Get the info for my tables in AGOL



In [4]:
cm_id = "03209b91e33f4c588007319d18cd6e70"
cm_item = gis.content.get(cm_id)

users_table = cm_item.tables[0]
inventory_table = cm_item.tables[1]
dependencies_table = cm_item.tables[2]

### Assemble users to be added to User table

In [38]:
# Get all users in the org
all_users = gis.users.search(query="*", max_users=10000)

# initialize empty dict to hold users info for AGOL table
users_list = []

# Loop through users; populate users dictionary
for user in all_users:

    # Initialize two dicts I need for each row to be added to users table
    update_dict = {}
    user_dict = {}

    # Add k/v pairs to the inner dict
    user_dict["username"] = user.username
    user_dict["user_email"] = user.email
    user_dict["last_login"] = user.lastLogin
    user_dict["days_since_last_login"] = get_time_delta(user.lastLogin)

    # Add inner dict to outer dict w/key "attributes"
    update_dict["attributes"] = user_dict

    # Append the user dict to the users list
    users_list.append(update_dict)

print(f"Number of users to add to table: {len(users_list)}")

print(users_list)

Number of users to add to table: 1
[{'attributes': {'username': 'AlderMaps', 'user_email': 'misti@aldermaps.com', 'last_login': 1758311377000, 'days_since_last_login': 0}}]


In [39]:
# Fake users data for use in creating Dashboard
# Because a (Personal Use License) org with a single user makes for a lame bar chart

ts = [1740205197408, 1726304097308, 1757318197908, 1748305167908, 1756218197908, 1755218197908]

def get_fake_users():

    users_list = [
        {
            "attributes": {
            "username": "Matt", 
            "user_email": "matt@aldermaps.com",
            "last_login": ts[0],
            "days_since_last_login": get_time_delta(ts[0])
            }
        },
        {
            "attributes": {
            "username": "Eric", 
            "user_email": "eric@aldermaps.com", 
            "last_login": ts[1],
            "days_since_last_login": get_time_delta(ts[1])
            }
        },
        {
            "attributes": {
            "username": "Alanda", 
            "user_email": "alanda@aldermaps.com", 
            "last_login": ts[2],
            "days_since_last_login": get_time_delta(ts[2])
            }
        },
        {
            "attributes": {
            "username": "Christine", 
            "user_email": "christine@aldermaps.com", 
            "last_login": ts[3],
            "days_since_last_login": get_time_delta(ts[3])
            }
        },
        {
            "attributes": {
            "username": "Khaled", 
            "user_email": "khaled@aldermaps.com", 
            "last_login": ts[4],
            "days_since_last_login": get_time_delta(ts[4])
            }
        },
        {
            "attributes": {
            "username": "Xiaoyi", 
            "user_email": "xiaoyi@aldermaps.com", 
            "last_login": ts[5],
            "days_since_last_login": get_time_delta(ts[5])
            }
        }
    ]

    return users_list

fake_users = get_fake_users()

# Tack my list of fake user dicts onto my own dict
# To disable addition of fake users, simply comment out the line below
users_list = users_list + fake_users

users_list

[{'attributes': {'username': 'AlderMaps',
   'user_email': 'misti@aldermaps.com',
   'last_login': 1758311377000,
   'days_since_last_login': 0}},
 {'attributes': {'username': 'Matt',
   'user_email': 'matt@aldermaps.com',
   'last_login': 1740205197408,
   'days_since_last_login': 209}},
 {'attributes': {'username': 'Eric',
   'user_email': 'eric@aldermaps.com',
   'last_login': 1726304097308,
   'days_since_last_login': 370}},
 {'attributes': {'username': 'Alanda',
   'user_email': 'alanda@aldermaps.com',
   'last_login': 1757318197908,
   'days_since_last_login': 11}},
 {'attributes': {'username': 'Christine',
   'user_email': 'christine@aldermaps.com',
   'last_login': 1748305167908,
   'days_since_last_login': 115}},
 {'attributes': {'username': 'Khaled',
   'user_email': 'khaled@aldermaps.com',
   'last_login': 1756218197908,
   'days_since_last_login': 24}},
 {'attributes': {'username': 'Xiaoyi',
   'user_email': 'xiaoyi@aldermaps.com',
   'last_login': 1755218197908,
   'days_s

### Write users to user table

In [40]:
# This script will effectively overwrite the output of the last run, 
# so before writing any new records we simply truncate tables
users_table.manager.truncate()

# Add users to table
users_update = users_table.edit_features(adds=users_list)

### Assemble item info to be added to Inventory table